In [1]:
import sys
import sys
sys.path.append(r"C:\My Projects\Health-Navigator")
print(sys.path)


['c:\\ProgramData\\anaconda3\\python312.zip', 'c:\\ProgramData\\anaconda3\\DLLs', 'c:\\ProgramData\\anaconda3\\Lib', 'c:\\ProgramData\\anaconda3', '', 'C:\\Users\\mamou\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\mamou\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32', 'C:\\Users\\mamou\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\mamou\\AppData\\Roaming\\Python\\Python312\\site-packages\\Pythonwin', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\Pythonwin', 'C:\\My Projects\\Health-Navigator']


In [2]:
import os
os.listdir(r"C:\My Projects\Health-Navigator")


['.git',
 '.gitignore',
 'app',
 'credentials.env',
 'gen-lang-client-0704427172-e146f940df9a.json',
 'LICENSE',
 'notes.txt',
 'README.md',
 'Workflow Diagram']

In [22]:
import sys
sys.path.append(r"C:\My Projects\Health-Navigator")


from langchain.agents import create_agent
from langchain.tools import tool
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import time
import os
load_dotenv(r'C:\My Projects\Health-Navigator\credentials.env')

from app.workflow.ml_models.numerical_models.heart_disease.heart_disease import predict_heart_disease
from app.workflow.vectordb.vectordb import HybridVectorDB

@tool
def predict_heart_disease_tool(
    HighBP: int,
    HighChol: int,
    BMI: float,
    Smoker: int,
    Stroke: int,
    Diabetes: int,
    PhysActivity: int,
    Fruits: int,
    Veggies: int,
    HvyAlcoholConsump: int,
    AnyHealthcare: int,
    GenHlth: int,
    MentHlth: int,
    PhysHlth: int,
    DiffWalk: int,
    Sex: int,
    Age: int,
    Education: int,
    Income: int,
    threshold: float = 0.40
) -> dict:
    """
    Predicts heart disease risk using a neural network model trained on patient health data.
    
    This tool analyzes 19 health-related features to estimate the probability of heart disease.
    Returns both a binary prediction (0=No Disease, 1=Disease) and the probability score.
    
    Args:
        HighBP (int): High blood pressure (0=No, 1=Yes)
        HighChol (int): High cholesterol (0=No, 1=Yes)
        BMI (float): Body Mass Index (continuous value, typically 12-98)
        Smoker (int): Smoking status (0=No, 1=Yes)
        Stroke (int): History of stroke (0=No, 1=Yes)
        Diabetes (int): Diabetes status (0=No, 1=Prediabetes, 2=Yes)
        PhysActivity (int): Physical activity in past 30 days (0=No, 1=Yes)
        Fruits (int): Consumes fruit 1+ times per day (0=No, 1=Yes)
        Veggies (int): Consumes vegetables 1+ times per day (0=No, 1=Yes)
        HvyAlcoholConsump (int): Heavy alcohol consumption (0=No, 1=Yes)
        AnyHealthcare (int): Has any healthcare coverage (0=No, 1=Yes)
        GenHlth (int): General health (1=Excellent to 5=Poor)
        MentHlth (int): Mental health not good for X days (0-30)
        PhysHlth (int): Physical health not good for X days (0-30)
        DiffWalk (int): Difficulty walking/climbing stairs (0=No, 1=Yes)
        Sex (int): Biological sex (0=Female, 1=Male)
        Age (int): Age category (1-13, where 1=18-24, 13=80+)
        Education (int): Education level (1-6, where 1=Never attended, 6=College graduate)
        Income (int): Income category (1-8, where 1=<$10k, 8=$75k+)
        threshold (float, optional): Probability threshold for positive prediction. Defaults to 0.40.
    
    Returns:
        dict: {
            'prediction': int (0 or 1, where 1 indicates heart disease),
            'probability': float (rounded to 4 decimal places, risk probability)
        }
    
    Example:
        result = predict_heart_disease_tool(
            HighBP=1, HighChol=1, BMI=28.5, Smoker=0, Stroke=0, Diabetes=0,
            PhysActivity=1, Fruits=1, Veggies=1, HvyAlcoholConsump=0,
            AnyHealthcare=1, GenHlth=3, MentHlth=5, PhysHlth=10, DiffWalk=0,
            Sex=1, Age=9, Education=5, Income=6
        )
        # Returns: {'prediction': 1, 'probability': 0.6234}
    """
    patient_data = {
        'HighBP': HighBP,
        'HighChol': HighChol,
        'BMI': BMI,
        'Smoker': Smoker,
        'Stroke': Stroke,
        'Diabetes': Diabetes,
        'PhysActivity': PhysActivity,
        'Fruits': Fruits,
        'Veggies': Veggies,
        'HvyAlcoholConsump': HvyAlcoholConsump,
        'AnyHealthcare': AnyHealthcare,
        'GenHlth': GenHlth,
        'MentHlth': MentHlth,
        'PhysHlth': PhysHlth,
        'DiffWalk': DiffWalk,
        'Sex': Sex,
        'Age': Age,
        'Education': Education,
        'Income': Income
    }
    
    return predict_heart_disease(patient_data, threshold=threshold)

@tool
def retrieve_from_vector_db(
    user_id: str,
    query: str,
    top_k: int = 100,
    filters: dict = None,
    date: str = None,
    date_filter: str = None
) -> list:
    """
    Retrieves relevant medical records and health documents from a user's personal vector database.
    
    Uses hybrid search (semantic + BM25) to find the most relevant information based on the query.
    Each user has their own isolated database identified by user_id.
    
    Args:
        user_id (str): Unique identifier for the user whose database to search
        query (str): Search query to find relevant documents
        top_k (int, optional): Number of results to return. Defaults to 100, to make sure you return all relevant results.
        filters (dict, optional): Metadata filters to apply, e.g., {'type': 'prescription', 'doctor': 'Dr. Smith'}
        date (str, optional): Date for filtering in 'YYYY-MM-DD' format
        date_filter (str, optional): How to filter by date - 'before', 'at', or 'after'. Requires date parameter, you can only pass these three values ('before', 'at', 'after').
    
    Returns:
        list: List of dictionaries, each containing:
            - text (str): The retrieved document text
            - metadata (dict): Document metadata (type, date, doctor, etc.)
            - score (float): Relevance score
    
    Example:
        # Basic search
        results = retrieve_from_vector_db(user_id="user123", query="blood pressure medications")
        
        # Search with filters and date
        results = retrieve_from_vector_db(
            user_id="user123",
            query="lab results",
            top_k=5,
            filters={'type': 'lab_report'},
            date="2024-01-01",
            date_filter="after"
        )
    """
    db = HybridVectorDB(user_id=user_id)
    
    return db.retrieve(
        query=query,
        top_k=top_k,
        filters=filters,
        date=date,
        date_filter=date_filter
    )


llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    google_api_key=os.environ["GOOGLE_API_KEY"],
)

# postgresql_uri = f'postgresql+psycopg2://{os.environ["POSTGRES_USERNAME"]}:{os.environ["POSTGRES_PASSWORD"]}@{os.environ["POSTGRES_HOST"]}:{os.environ["POSTGRES_PORT"]}/{os.environ["DATABASE_NAME"]}'
# db = SQLDatabase.from_uri(postgresql_uri)

mysql_uri = f'mysql+mysqlconnector://{os.environ["MYSQL_USERNAME"]}:{os.environ["MYSQL_PASSWORD"]}@{os.environ["MYSQL_HOST"]}:{os.environ["MYSQL_PORT"]}/{os.environ["DATABASE_NAME"]}'
db = SQLDatabase.from_uri(mysql_uri)

agent = None
current_user_id = None


def initialize_agent(user_id: str):
    global agent, current_user_id
    start_time = time.time()
    
    current_user_id = user_id

    system_prompt = f"""
    
You are a health data processing agent. You are ONE NODE in a larger health management workflow.

CRITICAL INSTRUCTIONS - READ CAREFULLY:

YOUR ROLE:
- You are NOT a conversational health assistant
- You are NOT an advice-giving chatbot
- You are a DATA PROCESSOR that analyzes input and produces structured output
- You receive input (user text, extracted document text, or both) and process it

YOUR RESPONSIBILITIES:
1. Analyze the input to determine what's needed
2. Use ML prediction models ONLY when the input contains or can provide the required features
3. Query databases (SQL and Vector) ONLY when you need data to feed ML models
4. Return a comprehensive, plain text output summarizing what was processed

IMPORTANT CONSTRAINTS:
- DO NOT give medical advice, health recommendations, or interpretations
- DO NOT have conversations or ask clarifying questions
- DO NOT add anything to databases - ONLY retrieve when necessary
- You may receive input that requires NO ML predictions and NO database queries - that's normal
- Sometimes you just pass through processed/formatted information

USER CONTEXT:
- Current user ID: "{user_id}"
- All database operations are scoped to this user

AVAILABLE TOOLS:
1. SQL Database Tools: Query structured health data (medications, appointments, lab results, etc.)
   - Use sql_db_list_tables to see available tables
   - Use sql_db_schema to understand table structure
   - Query data ONLY when needed to feed ML models

2. Vector Database (retrieve_from_vector_db): Search unstructured medical documents
   - Always use user_id: "{user_id}"
   - Retrieve ONLY when you need historical data for ML model input
   - Documents may contain: prescriptions, lab reports, doctor notes, health records

3. ML Model (predict_heart_disease_tool): Predict heart disease risk
   - Requires 19 specific features (HighBP, HighChol, BMI, Smoker, Stroke, Diabetes, PhysActivity, Fruits, Veggies, HvyAlcoholConsump, AnyHealthcare, GenHlth, MentHlth, PhysHlth, DiffWalk, Sex, Age, Education, Income)
   - Use ONLY when input contains or you can retrieve these features
   - Returns prediction (0 or 1) and probability

DECISION LOGIC:
- Does input contain health features? → Consider using ML model
- Do you need historical data for ML model? → Query databases
- Is input just information to process/format? → Process and return
- Missing required features for ML? → Don't force it, just process what you have

OUTPUT FORMAT:
- Return plain text ONLY
- Be comprehensive and detailed
- Include all processed information, predictions, and retrieved data
- Use \\n for line breaks
- NO tables, NO markdown formatting
- Report what you found/processed clearly

Current date/time: {time.strftime("%Y-%m-%d %H:%M:%S")} (YYYY-MM-DD HH:MM:SS)

Remember: You are a processing node, not a conversational agent. Analyze input → Use tools if needed → Return comprehensive output."""

    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    
    # Combine all tools
    all_tools = toolkit.get_tools() + [predict_heart_disease_tool, retrieve_from_vector_db]
    
    agent = create_agent(llm, all_tools, system_prompt=system_prompt)
    
    end_time = time.time()
    initialization_time = end_time - start_time
    print(f"Agent initialized in {initialization_time:.2f} seconds.")


def invoke_agent(user_input: str, user_id: str) -> str:
    """
    Process user input through the agent.
    
    Args:
        user_input: User query or extracted text from documents/images
        
    Returns:
        Comprehensive plain text output
    """


    if agent == None:
        initialize_agent(user_id=user_id)

    response = agent.invoke({
        "messages": [
            {"role": "user", "content": str(user_input)}
        ]
    })
    
    return response['messages'][-1].content

In [23]:
response = invoke_agent("I feel dizzy and have chest pain.", user_id="12345")

Agent initialized in 0.00 seconds.


As bm25s.BM25 requires k less than or equal to number of nodes added. Overriding the value of similarity_top_k to number of nodes added.
As bm25s.BM25 requires k less than or equal to number of nodes added. Overriding the value of similarity_top_k to number of nodes added.
As bm25s.BM25 requires k less than or equal to number of nodes added. Overriding the value of similarity_top_k to number of nodes added.


In [24]:
print(response)

[{'type': 'text', 'text': 'The following health-related information and historical records for user 12345 have been processed:\n\nCurrent Symptoms:\nThe user reported feeling dizzy and experiencing chest pain.\n\nRetrieved Medical Records Summary:\n- Laboratory Results (2023-11-03): Full blood panel indicated elevated cholesterol levels and slightly increased fasting glucose levels.\n- Imaging Report (2024-01-22): CT scan of the chest performed for shortness of breath showed mild progression of previously noted lung nodules.\n- Cardiac Imaging (2023-08-27): Echocardiogram showed stable left ventricular function with mild valve regurgitation.\n- Laboratory Results (2024-03-01 and 2025-03-01): Sequential blood tests showed declining hemoglobin levels, which may suggest developing anemia.\n- Abdominal Imaging (2022-09-18): Ultrasound revealed fatty liver changes, consistent with previous findings.\n- Neurological Imaging (2023-06-14): MRI brain scan showed an increase in the size of a ben

In [25]:
print(response[0]['text'])

The following health-related information and historical records for user 12345 have been processed:

Current Symptoms:
The user reported feeling dizzy and experiencing chest pain.

Retrieved Medical Records Summary:
- Laboratory Results (2023-11-03): Full blood panel indicated elevated cholesterol levels and slightly increased fasting glucose levels.
- Imaging Report (2024-01-22): CT scan of the chest performed for shortness of breath showed mild progression of previously noted lung nodules.
- Cardiac Imaging (2023-08-27): Echocardiogram showed stable left ventricular function with mild valve regurgitation.
- Laboratory Results (2024-03-01 and 2025-03-01): Sequential blood tests showed declining hemoglobin levels, which may suggest developing anemia.
- Abdominal Imaging (2022-09-18): Ultrasound revealed fatty liver changes, consistent with previous findings.
- Neurological Imaging (2023-06-14): MRI brain scan showed an increase in the size of a benign tumor in the frontal lobe.
- Ortho

In [7]:
def retrieve_from_vector_db(
    user_id: str,
    query: str,
    top_k: int = 100,
    filters: dict = None,
    date: str = None,
    date_filter: str = None
) -> list:
    """
    Retrieves relevant medical records and health documents from a user's personal vector database.
    
    Uses hybrid search (semantic + BM25) to find the most relevant information based on the query.
    Each user has their own isolated database identified by user_id.
    
    Args:
        user_id (str): Unique identifier for the user whose database to search
        query (str): Search query to find relevant documents
        top_k (int, optional): Number of results to return. Defaults to 100, to make sure you return all relevant results.
        filters (dict, optional): Metadata filters to apply, e.g., {'type': 'prescription', 'doctor': 'Dr. Smith'}
        date (str, optional): Date for filtering in 'YYYY-MM-DD' format
        date_filter (str, optional): How to filter by date - 'before', 'at', or 'after'. Requires date parameter, you can only pass these three values ('before', 'at', 'after').
    
    Returns:
        list: List of dictionaries, each containing:
            - text (str): The retrieved document text
            - metadata (dict): Document metadata (type, date, doctor, etc.)
            - score (float): Relevance score
    
    Example:
        # Basic search
        results = retrieve_from_vector_db(user_id="user123", query="blood pressure medications")
        
        # Search with filters and date
        results = retrieve_from_vector_db(
            user_id="user123",
            query="lab results",
            top_k=5,
            filters={'type': 'lab_report'},
            date="2024-01-01",
            date_filter="after"
        )
    """
    db = HybridVectorDB(user_id=user_id)
    
    return db.retrieve(
        query=query,
        top_k=top_k,
        filters=filters,
        date=date,
        date_filter=date_filter
    )

In [8]:
results = retrieve_from_vector_db("12345", "all blood tests")

As bm25s.BM25 requires k less than or equal to number of nodes added. Overriding the value of similarity_top_k to number of nodes added.


In [9]:
print(results)

[{'text': 'This is a new blood test, Blood test results from 2025-03-01 revealed declining hemoglobin levels relative to tests done six months earlier, suggesting developing anemia.', 'metadata': {'type': 'blood_test', 'test': 'hemoglobin', 'date': 20250301, 'category': 'lab', 'history': 'trend'}, 'score': 0.0994535519125683}, {'text': 'The patient completed a full blood panel on 2023-11-03. Results indicated elevated cholesterol and slightly increased fasting glucose levels compared to earlier tests.', 'metadata': {'type': 'blood_test', 'test': 'full_panel', 'date': 20231103, 'category': 'lab', 'history': 'trend'}, 'score': 0.09784946236559139}, {'text': 'Blood test results from 2024-03-01 revealed declining hemoglobin levels relative to tests done six months earlier, suggesting developing anemia.', 'metadata': {'type': 'blood_test', 'test': 'hemoglobin', 'date': 20240301, 'category': 'lab', 'history': 'trend'}, 'score': 0.09783183500793233}, {'text': 'A cardiac echocardiogram complet

# Testing from the agent.py (The code used in production)

In [1]:
from agent import invoke_agent

c:\ProgramData\anaconda3\Lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


resource module not available on Windows


In [2]:
response = invoke_agent("I feel dizzy and have chest pain.", user_id="12345")

Agent initialized in 0.18 seconds.


As bm25s.BM25 requires k less than or equal to number of nodes added. Overriding the value of similarity_top_k to number of nodes added.
As bm25s.BM25 requires k less than or equal to number of nodes added. Overriding the value of similarity_top_k to number of nodes added.
As bm25s.BM25 requires k less than or equal to number of nodes added. Overriding the value of similarity_top_k to number of nodes added.


In [3]:
print(response[0]['text'])

Based on the processing of the provided symptoms and the retrieval of historical health records for user 12345, the following information has been compiled:

Symptoms Reported:
The user is experiencing dizziness and chest pain.

Retrieved Historical Health Data:
- Lab Results (2023-11-03): Elevated cholesterol and slightly increased fasting glucose levels (indicative of prediabetes) were recorded.
- Cardiac History (2023-08-27): An echocardiogram showed stable left ventricular function with mild valve regurgitation.
- Respiratory/Imaging (2024-01-22): A chest CT scan performed for shortness of breath showed mild progression of previously identified lung nodules.
- Hematology (2024-03-01 & 2025-03-01): Blood tests indicated declining hemoglobin levels, suggestive of developing anemia.
- Other Findings:
  - Abdominal Ultrasound (2022-09-18): Fatty liver changes noted.
  - Brain MRI (2023-06-14): Benign tumor in the frontal lobe showed an increase in size.
  - PET Scan (2023-12-09): Incre